In [1]:
#初始化
import os
import toml
from openai import AzureOpenAI

# 读取 secrets.toml 文件
secrets = toml.load('secrets.toml')

client = AzureOpenAI(
  api_key = secrets.get('OPENAI_API_KEY'),
  api_version = secrets.get('AZURE_API_VERSION'),
  azure_endpoint = secrets.get('AZURE_API_BASE')
)

# 设置openai API默认参数, name = gpt-35-turbo-1106
def get_completion(prompt): 
    messages = [
        {"role": "system", "content": """Translate this markdown content to Simplified Chinese. Follow the below rules:
         1. Must keep original markdown format and position. 
         2. Do not change any markdown or html tags.
         3. The expected documentation or material shall keep the integrality of contents.
         4. The translated content shall be grammatically correct.
         5. The translated content shall be fluent and easy to understand."""},
        {"role": "user", "content": prompt}
        ]
    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages
      )
    return response.choices[0].message.content


In [11]:

filepath="output\China_Domestic_Substitution_and_Technological_indigenization_campaign_(xinchuang_initiative)\storm_gen_article_polished.txt"

# 获取文件夹路径
folder_path = os.path.dirname(filepath)

# 截取文件上一级目录名作为file_name
file_name = os.path.basename(folder_path)

print(folder_path)
print(file_name)

output\China_Domestic_Substitution_and_Technological_indigenization_campaign_(xinchuang_initiative)
China_Domestic_Substitution_and_Technological_indigenization_campaign_(xinchuang_initiative)


In [3]:
from markitdown import MarkItDown


md = MarkItDown()
result = md.convert(filepath)


# 定义要保存的文件路径, 将结果写入 Markdown 文件
output_filepath1 = os.path.join(os.path.dirname(filepath), 'non-translated_result.md')
with open(output_filepath1, 'w', encoding='utf-8') as file:
    file.write(result.text_content)

# 获取 completion 结果
completion_result = get_completion(result.text_content)

# 定义要保存的文件路径
output_filepath2 = os.path.join(os.path.dirname(filepath), 'translated_result.md')

# 将结果写入 Markdown 文件
with open(output_filepath2, 'w', encoding='utf-8') as file:
    file.write(completion_result)

print(f"Completion result saved to {output_filepath2}")

<>:3: DeprecationWarning: invalid escape sequence '\F'
<>:3: DeprecationWarning: invalid escape sequence '\F'
C:\Users\haxu\AppData\Local\Temp\ipykernel_11308\3202803042.py:3: DeprecationWarning: invalid escape sequence '\F'
  filepath="output\Forecast_China_equities_market_trend_in_2025\storm_gen_article_polished.txt"


Completion result saved to output\Forecast_China_equities_market_trend_in_2025\translated_result.md


In [13]:
#combine the translated result with references for EN and CN

import json
import os

# 文件路径
# folder_path = "How_do_you_measure_the_software_development_performance_gains_from_GitHub_Copilot"

# 读取 JSON 文件
with open(os.path.join(folder_path, 'url_to_info.json'), 'r', encoding='utf-8') as file:
    data = json.load(file)

# 提取标题，适应不同结构
# title = data.get('title') or data.get('info', {}).get('title') or 'default_title'
title = file_name

# 提取引用信息，使用可点击Markdown链接
references = []
for i, (url, info) in enumerate(data.get('url_to_info', {}).items(), start=1):
    ref_title = info.get('title', 'Untitled')
    if ref_title == "PDF" and 'meta' in info and 'query' in info['meta']:
        ref_title = info['meta']['query']
    references.append(f"- [{i}] [{ref_title}]({url})")

# 读取现有的 Markdown 文件内容
with open(os.path.join(folder_path, 'non-translated_result.md'), 'r', encoding='utf-8') as file:
    md_content = file.read()

# 将引用信息附加到 Markdown 文件末尾
output_filepath1 = os.path.join(folder_path, f"{title}_non-translated_result_with_references.md")

with open(output_filepath1, 'w', encoding='utf-8') as file:
    file.write(md_content)
    file.write("\n\n# references\n\n")
    for reference in references:
        file.write(f"{reference}\n")

# 读取现有的 Markdown 文件内容
with open(os.path.join(folder_path, 'translated_result.md'), 'r', encoding='utf-8') as file:
    md_content = file.read()

# 将引用信息附加到 Markdown 文件末尾
output_filepath = os.path.join(folder_path, f"{title}_translated_result_with_references.md")

with open(output_filepath, 'w', encoding='utf-8') as file:
    file.write(md_content)
    file.write("\n\n# 引用\n\n")
    for reference in references:
        file.write(f"{reference}\n")

print(f"References saved to {output_filepath}")

References saved to output\China_Domestic_Substitution_and_Technological_indigenization_campaign_(xinchuang_initiative)\China_Domestic_Substitution_and_Technological_indigenization_campaign_(xinchuang_initiative)_translated_result_with_references.md
